In [24]:
import pandas as pd
import math
from sklearn import preprocessing
import numpy as np
from sklearn.metrics import log_loss
import pandas as pd
from sklearn import preprocessing

In [25]:
# from google.colab import drive
# drive.mount('/content/drive')

In [26]:
data_path = "/home/vishnu/ML/challenges/AnalyticsOlympiad2022/data/"

train_data_path = data_path+"train.csv"
test_data_path = data_path+"test.csv"

train_data = pd.read_csv(train_data_path)
test_data = pd.read_csv(test_data_path)

del train_data['ID']
del train_data['POSTAL_CODE']


del test_data['ID']
del test_data['POSTAL_CODE']

### Normal Encoding

In [27]:

def label_encode_data(data):
    le = preprocessing.LabelEncoder()

    for col in data.columns:
        # print(col)
        if(col not in ('ID')):
        # print(col, train_data[col].dtypes)
            if(data[col].dtypes=='object'):
                le.fit(list(data[col]))
                data[col] = le.transform(list(data[col]))
    return data

### Data CLeaning and Feature Engineering

In [28]:

def one_hot_encode_data(data):
    le = preprocessing.LabelEncoder()

    for col in data.columns:
        # print(col)
        if(col not in ('ID')):
        # print(col, train_data[col].dtypes)
            if(data[col].dtypes=='object'):
                y = pd.get_dummies(data[col], prefix=col)

                data = data.drop(col, axis = 1)
                data = data.join(y)
    
    
    # del test_data['ID']
    
    return data

In [29]:
from sklearn.preprocessing import MinMaxScaler

def normalize_data(data):
    scaler = MinMaxScaler(feature_range=(0, 1))
    data['ANNUAL_MILEAGE'] = scaler.fit_transform(data['ANNUAL_MILEAGE'].values.reshape(-1,1))
    
    return data;

In [30]:
def create_new_features(data):
    data['VE_MA']= (data['VEHICLE_OWNERSHIP']+data['MARRIED']).astype(int)
    data['VE_CH']= (data['VEHICLE_OWNERSHIP']+data['CHILDREN']).astype(int)
    data['CH_MA']= (data['CHILDREN']+data['MARRIED']).astype(int)
    data['DUIS_ACC']= (data['DUIS']*data['PAST_ACCIDENTS']).astype(int)
    
    return data;

In [31]:
train_data = one_hot_encode_data(train_data)
# train_data = label_encode_data(train_data)
train_data = normalize_data(train_data)
train_data = create_new_features(train_data)

test_data = one_hot_encode_data(test_data)
# test_data = label_encode_data(test_data)
test_data = normalize_data(test_data)
test_data = create_new_features(test_data)



In [32]:
test_data.head()

,CREDIT_SCORE,VEHICLE_OWNERSHIP,MARRIED,CHILDREN,ANNUAL_MILEAGE,SPEEDING_VIOLATIONS,DUIS,PAST_ACCIDENTS,AGE_16-25,AGE_26-39,...,VEHICLE_YEAR_after 2015,VEHICLE_YEAR_before 2015,TYPE_OF_VEHICLE_HatchBack,TYPE_OF_VEHICLE_SUV,TYPE_OF_VEHICLE_Sedan,TYPE_OF_VEHICLE_Sports Car,VE_MA,VE_CH,CH_MA,DUIS_ACC
0,0.814707,1.0,0.0,1.0,0.421053,0,0,0,0,0,...,1,0,1,0,0,0,1,2,1,0
1,0.344051,1.0,0.0,0.0,0.578947,0,0,0,0,1,...,0,1,0,0,1,0,1,1,0,0
2,0.620691,0.0,1.0,0.0,0.526316,0,0,0,0,0,...,1,0,0,1,0,0,1,0,1,0
3,0.689658,1.0,1.0,1.0,0.736842,1,0,0,0,1,...,1,0,0,1,0,0,2,2,2,0
4,0.755666,0.0,0.0,1.0,0.631579,2,0,0,0,0,...,1,0,1,0,0,0,0,1,1,0


In [33]:
from sklearn.model_selection import train_test_split

target = train_data.OUTCOME
del train_data['OUTCOME']

In [34]:
X = train_data.values
y = target.values

# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.01, random_state=42)

In [38]:
X_train.shape

(103950, 35)

### Different model performance

In [36]:
# XGBClassifier(learning_rate=0.05,  n_estimators=200, max_depth=3, colsample_bytree = 0.5, gamma= 0.1, objective='binary:logistic', use_label_encoder=False)
# XGBoost validation Log loss =  0.6819472321905787
# submission score 0.68048

# XGBClassifier(learning_rate=0.05,  n_estimators=200, max_depth=3, colsample_bytree = 0.5, gamma= 0.1, objective='binary:logistic', use_label_encoder=False)
# one hot encoding
# submission score 0.68076


#XGBClassifier(learning_rate=0.007,  n_estimators=300, max_depth=6, colsample_bytree = 0.8, gamma= 0.1, objective='binary:logistic', use_label_encoder=False)
#XGBoost validation Log loss =  0.6819613814931067


#XGBClassifier(learning_rate=0.05, n_estimators=200, max_depth=2, colsample_bytree = 0.5, gamma= 0.05, use_label_encoder=False)
# submission score 0.68055

#XGBClassifier(learning_rate=0.05, n_estimators=100, max_depth=2, colsample_bytree = 0.6, gamma= 0.1, use_label_encoder=False)
# submission score 0.68048

#AdaBoostClassifier(n_estimators=200, learning_rate=0.01)
# submission score 0.68469

#XGBClassifier(learning_rate=0.05,  n_estimators=130, max_depth=3, colsample_bytree = 0.5, gamma= 0.1, objective='binary:logistic', use_label_encoder=False)
# submission score 0.68066

#XGBClassifier(learning_rate=0.05, n_estimators=102, max_depth=2, colsample_bytree = 0.6, gamma= 0.1, use_label_encoder=False)
#0.68052

### RandomForestClassifier

In [37]:
from sklearn.ensemble import RandomForestClassifier

RFModel = RandomForestClassifier(n_estimators=200, criterion='log_loss',verbose=5  )
RFModel.fit(X_train,y_train)

# Get the mean absolute error on the validation data
y_pred = RFModel.predict_proba(X_val)

ll = log_loss(y_val, y_pred)

print('Random forest validation Log Loss = ', ll)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


building tree 1 of 200
building tree 2 of 200


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.5s remaining:    0.0s


building tree 3 of 200
building tree 4 of 200


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.7s remaining:    0.0s


building tree 5 of 200
building tree 6 of 200
building tree 7 of 200
building tree 8 of 200
building tree 9 of 200
building tree 10 of 200
building tree 11 of 200
building tree 12 of 200
building tree 13 of 200
building tree 14 of 200
building tree 15 of 200
building tree 16 of 200
building tree 17 of 200
building tree 18 of 200
building tree 19 of 200
building tree 20 of 200
building tree 21 of 200
building tree 22 of 200
building tree 23 of 200
building tree 24 of 200
building tree 25 of 200
building tree 26 of 200
building tree 27 of 200
building tree 28 of 200
building tree 29 of 200
building tree 30 of 200
building tree 31 of 200
building tree 32 of 200
building tree 33 of 200
building tree 34 of 200
building tree 35 of 200
building tree 36 of 200
building tree 37 of 200
building tree 38 of 200
building tree 39 of 200
building tree 40 of 200
building tree 41 of 200
building tree 42 of 200
building tree 43 of 200
building tree 44 of 200
building tree 45 of 200
building tree 46 of 2

[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:   37.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    0.1s finished


In [39]:

testX = test_data.values

y_test_pred = RFModel.predict_proba(testX)

# y_test_pred = xgb.predict_proba(val)
y_test_pred = y_test_pred[:,1]
y_test_pred

submission_df_XG = pd.DataFrame(y_test_pred, columns=['OUTCOME'])
submission_df_XG.to_csv('results/sub_oh_RF2_200.csv', index=False)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    3.0s finished


### XGBClasifier

In [60]:
# !pip install xgboost==1.5 


#### XGB train

In [109]:
import sklearn
from sklearn import metrics
from sklearn import svm, datasets

import xgboost
from xgboost import XGBClassifier


XGBModel = XGBClassifier(learning_rate=0.05, n_estimators=102, max_depth=2, colsample_bytree = 0.6, gamma= 0.1, use_label_encoder=False)


eval_set = [(X_val, y_val)]
XGBModel.fit(X_train, y_train, eval_metric="logloss", eval_set=eval_set, verbose=True)

# XGBModel.fit(X,y, eval_metric="logloss", verbose=True)


[0]	validation_0-logloss:0.69185
[1]	validation_0-logloss:0.69065
[2]	validation_0-logloss:0.68956
[3]	validation_0-logloss:0.68856
[4]	validation_0-logloss:0.68768
[5]	validation_0-logloss:0.68691
[6]	validation_0-logloss:0.68621
[7]	validation_0-logloss:0.68557
[8]	validation_0-logloss:0.68499
[9]	validation_0-logloss:0.68445
[10]	validation_0-logloss:0.68395
[11]	validation_0-logloss:0.68350
[12]	validation_0-logloss:0.68310
[13]	validation_0-logloss:0.68273
[14]	validation_0-logloss:0.68240
[15]	validation_0-logloss:0.68209
[16]	validation_0-logloss:0.68183
[17]	validation_0-logloss:0.68161
[18]	validation_0-logloss:0.68138
[19]	validation_0-logloss:0.68115
[20]	validation_0-logloss:0.68093
[21]	validation_0-logloss:0.68076
[22]	validation_0-logloss:0.68058
[23]	validation_0-logloss:0.68046
[24]	validation_0-logloss:0.68034
[25]	validation_0-logloss:0.68021
[26]	validation_0-logloss:0.68010
[27]	validation_0-logloss:0.67997
[28]	validation_0-logloss:0.67990
[29]	validation_0-loglos

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.6,
              enable_categorical=False, gamma=0.1, gpu_id=-1,
              importance_type=None, interaction_constraints='',
              learning_rate=0.05, max_delta_step=0, max_depth=2,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=102, n_jobs=4, num_parallel_tree=1, predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              subsample=1, tree_method='exact', use_label_encoder=False,
              validate_parameters=1, verbosity=None)

In [110]:
# Get the mean absolute error on the validation data :
XGBpredictions = XGBModel.predict_proba(X_val)

rmse = log_loss(y_val, XGBpredictions)
print('XGBoost validation Log loss = ', rmse)

#XGBoost validation Log loss =  0.6819472321905787




XGBoost validation Log loss =  0.6790449727149237


#### XGB predict

In [111]:

testX = test_data.values

y_test_pred = XGBModel.predict_proba(testX)

# y_test_pred = xgb.predict_proba(val)
y_test_pred = y_test_pred[:,1]
y_test_pred

submission_df_XG = pd.DataFrame(y_test_pred, columns=['OUTCOME'])
submission_df_XG.to_csv('results/sub_no_pscode_XG_103.csv', index=False)


### Adaboost

In [29]:
from sklearn.ensemble import AdaBoostClassifier


adaboostcls = AdaBoostClassifier(n_estimators=200, learning_rate=0.01)


ADBModel = adaboostcls.fit(X, y)



In [49]:

testX = test_data.values

y_test_pred_proba = ADBModel.predict_proba(testX)

# y_test_pred = xgb.predict_proba(val)
y_test_pred = y_test_pred[:,1]
y_test_pred

submission_df = pd.DataFrame(y_test_pred, columns=['OUTCOME'])
submission_df.to_csv('results/submission_df_ADB_2001.csv', index=False)


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [51]:
y_test_pred_proba

array([[0.53653452, 0.46346548],
       [0.5301473 , 0.4698527 ],
       [0.53653452, 0.46346548],
       ...,
       [0.53653452, 0.46346548],
       [0.53225436, 0.46774564],
       [0.53321609, 0.46678391]])

In [52]:
y_test_pred_proba[:,1]

array([0.46346548, 0.4698527 , 0.46346548, ..., 0.46346548, 0.46774564,
       0.46678391])

In [53]:
y_test_pred = ADBModel.predict(testX)


In [54]:
y_test_pred

array([0., 0., 0., ..., 0., 0., 0.])

### Neural Net

In [14]:

from keras import backend as K
import keras
import numpy as np
from tensorflow.python.ops import math_ops

from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten

n_inputs, n_outputs = X.shape[1], 1

NN_model = Sequential()
NN_model.add(Dense(32, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))


# The Hidden Layers :
NN_model.add(Dense(64, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(128, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(128, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(64, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(32, kernel_initializer='normal',activation='relu'))

NN_model.add(Dense(n_outputs, activation='sigmoid'))
NN_model.compile(loss='binary_crossentropy', optimizer='adam')

# # The Output Layer :
# NN_model.add(Dense(1, kernel_initializer='normal',activation='sigmoid'))

# # Compile the network :
# NN_model.compile(loss=log_loss, metrics=['mean_absolute_error'])
NN_model.summary()

2022-10-29 11:53:11.755505: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-10-29 11:53:11.755550: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-10-29 11:53:11.755581: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (vishnu-Lenovo-G50-80): /proc/driver/nvidia/version does not exist
2022-10-29 11:53:11.755941: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                1056      
                                                                 
 dense_1 (Dense)             (None, 64)                2112      
                                                                 
 dense_2 (Dense)             (None, 128)               8320      
                                                                 
 dense_3 (Dense)             (None, 256)               33024     
                                                                 
 dense_4 (Dense)             (None, 128)               32896     
                                                                 
 dense_5 (Dense)             (None, 64)                8256      
                                                                 
 dense_6 (Dense)             (None, 32)                2

In [15]:
NN_model.fit(X, y, epochs=150, batch_size=64, validation_split = 0.2, callbacks=None)

Epoch 1/150
1313/1313 [==============================] - 5s 4ms/step - loss: 0.7024 - val_loss: 0.6806
Epoch 2/150
1313/1313 [==============================] - 5s 4ms/step - loss: 0.6822 - val_loss: 0.6805
Epoch 3/150
1313/1313 [==============================] - 4s 3ms/step - loss: 0.6816 - val_loss: 0.6804
Epoch 4/150
1313/1313 [==============================] - 5s 3ms/step - loss: 0.6814 - val_loss: 0.6805
Epoch 5/150
1313/1313 [==============================] - 5s 4ms/step - loss: 0.6813 - val_loss: 0.6806
Epoch 6/150
1313/1313 [==============================] - 7s 5ms/step - loss: 0.6814 - val_loss: 0.6804
Epoch 7/150
1313/1313 [==============================] - 8s 6ms/step - loss: 0.6814 - val_loss: 0.6804
Epoch 8/150
1313/1313 [==============================] - 4s 3ms/step - loss: 0.6814 - val_loss: 0.6804
Epoch 9/150
1313/1313 [==============================] - 4s 3ms/step - loss: 0.6814 - val_loss: 0.6804
Epoch 10/150
1313/1313 [==============================] - 4s 3ms/step - l

In [16]:

testX = test_data.values

predictions = NN_model.predict(testX)


1407/1407 [==============================] - 2s 1ms/step


In [18]:
submission_df_NN = pd.DataFrame(predictions)
submission_df_NN.columns = ["OUTCOME"]
submission_df_NN.to_csv('results/submission_oh_NN.csv', index=False)


,CREDIT_SCORE,VEHICLE_OWNERSHIP,MARRIED,CHILDREN,POSTAL_CODE,ANNUAL_MILEAGE,SPEEDING_VIOLATIONS,DUIS,PAST_ACCIDENTS,AGE_16-25,...,INCOME_middle class,INCOME_poverty,INCOME_upper class,INCOME_working class,VEHICLE_YEAR_after 2015,VEHICLE_YEAR_before 2015,TYPE_OF_VEHICLE_HatchBack,TYPE_OF_VEHICLE_SUV,TYPE_OF_VEHICLE_Sedan,TYPE_OF_VEHICLE_Sports Car
0,0.814707,1.0,0.0,1.0,10238,10000.0,0,0,0,0,...,0,0,0,1,1,0,1,0,0,0
1,0.344051,1.0,0.0,0.0,34073,13000.0,0,0,0,0,...,0,0,1,0,0,1,0,0,1,0
2,0.620691,0.0,1.0,0.0,10238,12000.0,0,0,0,0,...,0,0,1,0,1,0,0,1,0,0
3,0.689658,1.0,1.0,1.0,10238,16000.0,1,0,0,0,...,0,0,1,0,1,0,0,1,0,0
4,0.755666,0.0,0.0,1.0,92101,14000.0,2,0,0,0,...,1,0,0,0,1,0,1,0,0,0


### END